In [ ]:
# Imports
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from langdetect import detect
from pydantic import BaseModel
import datetime
import your_information

# Webscraping 
Get job description text from the URL of the job posting using `Requests` and `BeautifulSoup`.

In [ ]:
# Send HTTP GET request
url_response = requests.get(your_information.job_description_url)

# Extract HTML content
html_content = url_response.text

# Initialize BeautifulSoup object to parse HTML content
soup = BeautifulSoup(html_content, "html.parser")

# Extract the text
job_description = soup.get_text()

# Remove whitespace characters
job_description = " ".join(job_description.split())  

# Show job description text
print(job_description)

Automatically detect language of job description using `langdetect`.

In [ ]:
# Detect language of job description 
# Note: Returns ISO 639-1 language codes (e.g., 'en' for English or 'de' for German)
language = detect(job_description)

# Show detected language
print(f"Detected language: {language}")

# Information Extraction 
Get relevant information from job description text in JSON format using `OpenAI's chat completions API` with `response_format` parameter for `structured outputs`. Leverage the `Pydantic` library to define a clear and robust JSON schema for validating and structuring the extracted data.

In [ ]:
# Initialize OpenAI client to connect to ChatGPT API
client = OpenAI(api_key=your_information.OPENAI_API_KEY)

# Define JSON output format
class JobDescriptionExtraction(BaseModel):
    employer: str
    job_title: str
    requirements: list[str]
    tasks: list[str]
    contact_person: str
    address: str

# Define system prompt for information extraction in English and German
extraction_system_prompt = {
    "en": """
    You are an expert at structured data extraction. You will be given unstructured text from a job description 
    and should convert it into the given structure. If the information is missing in the job description, use 
    "unknown" as the value. Respond as concisely as possible.
    """,
    "de": """
    Du bist ein Experte für die Extraktion strukturierter Daten. Dir wird unstrukturierter Text aus einer 
    Stellenbeschreibung gegeben, den du in das vorgegebene Format umwandeln sollst. Wenn die Information 
    in der Stellenbeschreibung fehlt, verwende "unbekannt" als Wert. Antworte so kurz wie möglich.
    """
}

# Make API request to extract information from job description text
extraction_response_raw = client.beta.chat.completions.parse(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": extraction_system_prompt[language]},  
        {"role": "user", "content": job_description}
    ],
    response_format = JobDescriptionExtraction 
)

# Show raw extraction response
extraction_response_raw

In [ ]:
# Get the job information from the parsed extraction response and convert it to a dict
job_information = extraction_response_raw.choices[0].message.parsed.dict()
job_information

# Cover Letter Generation 
Create 3 cover letter suggestions by matching qualifications with job requirements using `OpenAI's chat completions API`.

In [ ]:
# Helper function: Get ChatGPT response from text prompt using API
def get_completion(prompt, system_prompt="", model="gpt-4o-mini", temperature=0.7): 
    response = client.chat.completions.create(
        model = model,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature = temperature  # controls randomness of generated text with values from 0 (little to no randomness) to 1 (maximum randomness)
    )
    return response

In [ ]:
# System prompt for cover letter generation in English and German
cover_letter_system_prompt = {
    "en": """
    You are an expert assistant in creating professional cover letters. 
    Your job is to generate precise, compact, and professional cover letters in English. 
    The tone should be formal and professional, with appropriate attention to detail and presentation.
    """,
    "de": """
    Du bist ein Experte im Erstellen professioneller Bewerbungsanschreiben.
    Deine Aufgabe ist es, präzise, kompakte und professionelle Anschreiben auf Deutsch zu erstellen.
    Der Tonfall soll formell und professionell sein, mit angemessener Aufmerksamkeit für Details und Präsentation.
    """
}

# Cover letter prompt in English and German
cover_letter_prompt = {
    "en": f"""
    Your task is to create a professional cover letter.
    
    Address the letter to the following employer, address, job position, and contact person:
    Employer: {job_information["employer"]}
    Address: {job_information["address"]}
    Job Position: {job_information["job_title"]}
    Contact Person: {job_information["contact_person"]}
    
    Use the following sender information:
    Name: {your_information.name}
    Address: {your_information.address}
    Phone Number: {your_information.phone}
    Email: {your_information.email}
    
    Include the location and date in the letterhead of the cover letter.
    Use the location from: {your_information.address}
    Use the current date in the English date format: {datetime.date.today()}
    
    Describe how the education, work experience, skills, and motivation fulfill 
    the job requirements and tasks. Use the following information:
    Requirements: {job_information["requirements"]}
    Tasks: {job_information["tasks"]}
    Education: {your_information.education}
    Work Experience: {your_information.work_experience}
    Skills: {your_information.skills}
    Motivation: {your_information.motivation}
    
    State the salary expectations and possible start date as follows:
    Salary Expectations: {your_information.salary_expectations}
    Possible Start Date: {your_information.possible_start_date}
    
    Write in a professional, concise, and compact tone.
    
    Sign the cover letter as {your_information.name}.
    """,
    "de": f"""
    Deine Aufgabe ist es, ein professionelles Bewerbungsanschreiben zu erstellen.
    
    Adressiere das Anschreiben an folgenden Arbeitgeber, Adresse, Stelle und Kontaktperson:
    Arbeitgeber: {job_information["employer"]}
    Adresse: {job_information["address"]}
    Stelle: {job_information["job_title"]}
    Kontaktperson: {job_information["contact_person"]}
    
    Verwende folgenden Absender:
    Name: {your_information.name}
    Adresse: {your_information.address}
    Telefonnummer: {your_information.phone}
    E-Mail: {your_information.email}
    
    Verwende im Briefkopf des Anschreibens Ort und Datum. 
    Verwende den Ort aus: {your_information.address}
    Verwende das folgende, aktuelle Datum im deutschen Datumsformat: {datetime.date.today()}
    
    Beschreibe inwiefern die Anforderungen und Aufgaben erfüllt werden durch die 
    Ausbildung, Arbeitserfahrung, Kompetenzen und Motivation. 
    Verwende hierfür die folgenden Informationen:
    Anforderungen: {job_information["requirements"]}
    Aufgaben: {job_information["tasks"]}
    Ausbildung: {your_information.education}
    Arbeitserfahrung: {your_information.work_experience}
    Kompetenzen: {your_information.skills}
    Motivation: {your_information.motivation}
    
    Nenne folgende Gehaltsvorstellung und mögliches Eintrittsdatum:
    Gehaltsvorstellung : {your_information.salary_expectations}
    Mögliches Eintrittsdatum: {your_information.possible_start_date}
    
    Schreibe in einem professionellen, präzisen und kompakten Ton.
    
    Unterschreibe das Anschreiben als {your_information.name}.
    """
}

# Make API requests to create 3 cover letter suggestions
cover_letter_ls = []
for i in range(3):
    # Generate a single cover letter
    cover_letter_response = get_completion(prompt=cover_letter_prompt[language],  
                                           system_prompt=cover_letter_system_prompt[language]  
                                          ).choices[0].message.content 
    # Remove ** symbols
    cover_letter_response = cover_letter_response.replace('**', '')
    # Append cover letter to list
    cover_letter_ls.append(cover_letter_response)

# Show cover letter suggestions
for i in range(len(cover_letter_ls)):
    print(f"Anschreiben {i+1}")
    print("=" * 5)
    print(cover_letter_ls[i])
    print("=" * 30)